In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization, Concatenate
from tensorflow.keras.models import Model

# Define the generator
def build_generator():
    inputs = Input(shape=(128, 128, 3))
    
    # Encoder
    e1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    e1 = LeakyReLU(alpha=0.2)(e1)
    
    e2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(e1)
    e2 = BatchNormalization()(e2)
    e2 = LeakyReLU(alpha=0.2)(e2)
    
    # Decoder
    d1 = Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(e2)
    d1 = BatchNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    
    d2 = Conv2DTranspose(3, kernel_size=4, strides=2, padding='same')(d1)
    outputs = tf.nn.tanh(d2)
    
    return Model(inputs, outputs)

# Define the discriminator
def build_discriminator():
    inputs = Input(shape=(128, 128, 3))
    
    d1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    d1 = LeakyReLU(alpha=0.2)(d1)
    
    d2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(d1)
    d2 = BatchNormalization()(d2)
    d2 = LeakyReLU(alpha=0.2)(d2)
    
    d3 = Conv2D(256, kernel_size=4, strides=2, padding='same')(d2)
    d3 = BatchNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    
    outputs = Conv2D(1, kernel_size=4, strides=1, padding='same')(d3)
    
    return Model(inputs, outputs)

# Build and compile the GAN
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])
    discriminator.trainable = False
    
    z = Input(shape=(128, 128, 3))
    img = generator(z)
    
    validity = discriminator(img)
    
    return Model(z, validity)

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))


2024-06-05 11:17:06.664804: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 11:17:06.664908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 11:17:06.793566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [1]:
import matplotlib.pyplot as plt

import numpy as np

# Load your data here
damaged_images = '/kaggle/input/fyp1000-128-masked2/'
original_images = '/kaggle/input/fyp1000-128p/'

# Training parameters
epochs = 10000
batch_size = 4
save_interval = 100

# Labels for real and fake images
valid = np.ones((batch_size, 16, 16, 1))
fake = np.zeros((batch_size, 16, 16, 1))


# Initialize lists to store losses
d_losses = []
g_losses = []

def sample_images(epoch):
    r, c = 2, 5
    idx = np.random.randint(0, damaged_images.shape[0], 10)
    imgs_damaged = damaged_images[idx]
    gen_imgs = generator.predict(imgs_damaged)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"images/{epoch}.png")
    plt.close()

def plot_losses(epoch):
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label="Discriminator loss")
    plt.plot(g_losses, label="Generator loss")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Generator and Discriminator Losses')
    plt.savefig(f"losses/losses_epoch_{epoch}.png")
    plt.close()

for epoch in range(epochs):
    idx = np.random.randint(0, damaged_images.shape[0], batch_size)
    imgs_damaged = damaged_images[idx]
    imgs_original = original_images[idx]

    # Generate a batch of images
    gen_imgs = generator.predict(imgs_damaged)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs_original, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    g_loss = gan.train_on_batch(imgs_damaged, valid)

    # Save the losses
    d_losses.append(d_loss[0])
    g_losses.append(g_loss)

    # Print progress
    print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

    # Save generated image samples and plot losses every 500 epochs
    if epoch % 500 == 0:
        sample_images(epoch)
        plot_losses(epoch)
        generator.save_weights(f'weights/generator_weights_epoch_{epoch}.h5')
        discriminator.save_weights(f'weights/discriminator_weights_epoch_{epoch}.h5')


AttributeError: 'str' object has no attribute 'shape'

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model

# Define the generator
def build_generator():
    inputs = Input(shape=(128, 128, 3))
    
    # Encoder
    e1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    e1 = LeakyReLU(alpha=0.2)(e1)
    
    e2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(e1)
    e2 = BatchNormalization()(e2)
    e2 = LeakyReLU(alpha=0.2)(e2)
    
    # Decoder
    d1 = Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(e2)
    d1 = BatchNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    
    d2 = Conv2DTranspose(3, kernel_size=4, strides=2, padding='same')(d1)
    outputs = tf.keras.activations.tanh(d2)
    
    return Model(inputs, outputs)

# Define the discriminator
def build_discriminator():
    inputs = Input(shape=(128, 128, 3))
    
    d1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    d1 = LeakyReLU(alpha=0.2)(d1)
    
    d2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(d1)
    d2 = BatchNormalization()(d2)
    d2 = LeakyReLU(alpha=0.2)(d2)
    
    d3 = Conv2D(256, kernel_size=4, strides=2, padding='same')(d2)
    d3 = BatchNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    
    outputs = Conv2D(1, kernel_size=4, strides=1, padding='same')(d3)
    
    return Model(inputs, outputs)

# Build and compile the GAN
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])
    discriminator.trainable = False
    
    z = Input(shape=(128, 128, 3))
    img = generator(z)
    
    validity = discriminator(img)
    
    return Model(z, validity)

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))


In [9]:
import os
def load_images_from_folder(folder, image_size=(128, 128), num_images=300):
    images = []
    filenames = sorted(os.listdir(folder))[:num_images]  # Sort and take only the top 300 filenames
    for filename in filenames:
        img_path = os.path.join(folder, filename)
        try:
            img = Image.open(img_path).convert('RGB')  # Convert to RGB to handle grayscale images
            img = img.resize(image_size)
            img = np.array(img)
            if img.shape == (128, 128, 3):  # Ensure the image is RGB
                images.append(img)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    return np.array(images)

In [10]:
import numpy as np
import matplotlib.pyplot as plt

# Load your data here
damaged_images_path = '/kaggle/input/fyp1000-128-masked2/'
original_images_path = '/kaggle/input/fyp1000-128p/'

damaged_images = load_images_from_folder(damaged_images_path)
original_images = load_images_from_folder(original_images_path)

print("damaged images shape: ")
print(damaged_images.shape[0])
print("Original images shape: ", original_images.shape[0])

# Training parameters
epochs = 200
batch_size = 15
save_interval = 50

# Labels for real and fake images
valid = np.ones((batch_size, 16, 16, 1))
fake = np.zeros((batch_size, 16, 16, 1))

os.makedirs('images', exist_ok=True)
os.makedirs('losses', exist_ok=True)
os.makedirs('weights', exist_ok=True)

# Initialize lists to store losses
d_losses = []
g_losses = []

def sample_images(epoch):
    r, c = 2, 5
    idx = np.random.randint(0, damaged_images.shape[0], 10)
    imgs_damaged = damaged_images[idx]
    gen_imgs = generator.predict(imgs_damaged)

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"images/{epoch+1}.jpg")
    plt.close()

def plot_losses(epoch):
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label="Discriminator loss")
    plt.plot(g_losses, label="Generator loss")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Generator and Discriminator Losses')
    plt.savefig(f"losses/losses_epoch_{epoch}.jpg")
    plt.close()

# Training parameters
epochs = 2000
batch_size = 32
save_interval = 500

# Labels for real and fake images
valid = np.ones((batch_size, 16, 16, 1))
fake = np.zeros((batch_size, 16, 16, 1))

for epoch in range(epochs):
    idx = np.random.randint(0, damaged_images.shape[0], batch_size)
    imgs_damaged = damaged_images[idx]
    imgs_original = original_images[idx]

    # Generate a batch of images
    gen_imgs = generator.predict(imgs_damaged)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(imgs_original, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    g_loss = gan.train_on_batch(imgs_damaged, valid)

    # Save the losses
    d_losses.append(d_loss[0])
    g_losses.append(g_loss)

    # Print progress
    print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

    # Save generated image samples and plot losses every 500 epochs
    if epoch % save_interval == 0:
        sample_images(epoch)
        plot_losses(epoch)
        generator.save_weights(f'weights/generator_weights_epoch_{epoch}.weights.h5')
        discriminator.save_weights(f'weights/discriminator_weights_epoch_{epoch}.weights.h5')


Error loading image /kaggle/input/fyp1000-128-masked2/1.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/10.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/100.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/1000.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/101.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/102.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/103.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/104.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/105.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/106.jpg: name 'Image' is not defined
Error loading image /kaggle/input/fyp1000-128-masked2/107.jpg: name 'Image' is not defined
E

ValueError: high <= 0

In [1]:
#import tensorflow as tf
import os
from PIL import Image

def load_image(image_path, image_size=(128, 128)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = (image / 127.5) - 1  # Normalize to [-1, 1]
    return image

def load_data(damaged_images_path, original_images_path):
    damaged_images_files = sorted([os.path.join(damaged_images_path, file) for file in os.listdir(damaged_images_path)])
    original_images_files = sorted([os.path.join(original_images_path, file) for file in os.listdir(original_images_path)])
    
    dataset = tf.data.Dataset.from_tensor_slices((damaged_images_files, original_images_files))
    
    def load_and_preprocess(damaged_image_path, original_image_path):
        damaged_image = load_image(damaged_image_path)
        original_image = load_image(original_image_path)
        return damaged_image, original_image
    
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset


In [4]:
import tensorflow as tf
# Load your data here
damaged_images_path = '/kaggle/input/fyp1000-128-masked2/'
original_images_path = '/kaggle/input/fyp1000-128p/'


batch_size = 32
epochs = 10000
save_interval = 500

dataset = load_data(damaged_images_path, original_images_path)
dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)


2024-06-05 12:45:35.393830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-05 12:45:35.393934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-05 12:45:35.515642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model

# Define the generator
def build_generator():
    inputs = Input(shape=(128, 128, 3))
    
    # Encoder
    e1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    e1 = LeakyReLU(alpha=0.2)(e1)
    
    e2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(e1)
    e2 = BatchNormalization()(e2)
    e2 = LeakyReLU(alpha=0.2)(e2)
    
    # Decoder
    d1 = Conv2DTranspose(64, kernel_size=4, strides=2, padding='same')(e2)
    d1 = BatchNormalization()(d1)
    d1 = LeakyReLU(alpha=0.2)(d1)
    
    d2 = Conv2DTranspose(3, kernel_size=4, strides=2, padding='same')(d1)
    outputs = tf.keras.activations.tanh(d2)
    
    return Model(inputs, outputs)

# Define the discriminator
def build_discriminator():
    inputs = Input(shape=(128, 128, 3))
    
    d1 = Conv2D(64, kernel_size=4, strides=2, padding='same')(inputs)
    d1 = LeakyReLU(alpha=0.2)(d1)
    
    d2 = Conv2D(128, kernel_size=4, strides=2, padding='same')(d1)
    d2 = BatchNormalization()(d2)
    d2 = LeakyReLU(alpha=0.2)(d2)
    
    d3 = Conv2D(256, kernel_size=4, strides=2, padding='same')(d2)
    d3 = BatchNormalization()(d3)
    d3 = LeakyReLU(alpha=0.2)(d3)
    
    outputs = Conv2D(1, kernel_size=4, strides=1, padding='same')(d3)
    
    return Model(inputs, outputs)

# Build and compile the GAN
def build_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), metrics=['accuracy'])
    discriminator.trainable = False
    
    z = Input(shape=(128, 128, 3))
    img = generator(z)
    
    validity = discriminator(img)
    
    return Model(z, validity)

generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0002, 0.5))


/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [9]:
import matplotlib.pyplot as plt
import os
import numpy as np

#print("damaged images shape: ")
#print(damaged_images.shape[0])
#print("Original images shape: ", original_images.shape[0])

# Training parameters
epochs = 10000
batch_size = 32
save_interval = 1000

# Labels for real and fake images
valid = np.ones((batch_size, 16, 16, 1))
fake = np.zeros((batch_size, 16, 16, 1))

# Create directories if they do not exist
os.makedirs('images', exist_ok=True)
os.makedirs('losses', exist_ok=True)
os.makedirs('weights', exist_ok=True)

# Initialize lists to store losses
d_losses = []
g_losses = []

def sample_images(epoch, generator, dataset):
    r, c = 2, 5
    imgs_damaged, _ = next(iter(dataset.take(1)))
    gen_imgs = generator.predict(imgs_damaged[:10])

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow((gen_imgs[cnt] * 0.5) + 0.5)  # Rescale images from [-1, 1] to [0, 1]
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"images/{epoch+1}.jpg")
    plt.close()

def plot_losses(epoch):
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label="Discriminator loss")
    plt.plot(g_losses, label="Generator loss")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Generator and Discriminator Losses')
    plt.savefig(f"losses/losses_epoch_{epoch+1}.png")
    plt.close()

# Labels for real and fake images
valid = np.ones((batch_size, 16, 16, 1))
fake = np.zeros((batch_size, 16, 16, 1))

for epoch in range(epochs):
    for batch, (imgs_damaged, imgs_original) in enumerate(dataset):
        # Generate a batch of images
        gen_imgs = generator.predict(imgs_damaged)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(imgs_original, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        g_loss = gan.train_on_batch(imgs_damaged, valid)

        # Save the losses
        d_losses.append(d_loss[0])
        g_losses.append(g_loss)

        # Print progress
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100*d_loss[1]}] [G loss: {g_loss}]")

    # Save generated image samples and plot losses every 500 epochs
    if epoch % save_interval == 0:
        sample_images(epoch, generator, dataset)
        plot_losses(epoch)
        generator.save_weights(f'weights/generator_weights_epoch_{epoch+1}.h5')
        discriminator.save_weights(f'weights/discriminator_weights_epoch_{epoch+1}.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


ValueError: Dimensions must be equal, but are 8 and 32 for '{{node Equal}} = Equal[T=DT_DOUBLE, incompatible_shape_error=true](data_1, Cast_2)' with input shapes: [8,16,16,1], [32,16,16,1].